In [ ]:
import re

def get_floats(string):
    """
    Gibt alle Kommazahlen des übergebenen Strings als float-Liste zurück.
    Diese Funktion wird benötigt, wenn wir unsere Datenbank neu aufsetzen sollten und im gleichen Zuge die Positionen im GIS-Format speichern werden.
    
    Eingabebeispiel:
    string = POINT (51.375887 7.660134)
    
    Ausgabebeispiel:
    [51.375887, 7.660134]
    
    """
    
    return [float(i) for i in re.findall("\d+\.\d+", string)]


In [ ]:
from math import sin, cos, sqrt, atan2, radians, pi

def calculate_distance(orig_point, dest_point):
    """
    Gibt den Abstand zwischen den beiden übergebenen Punkten in Kilometern als float zurück.
    
    Eingabebeispiel:
    orig_point = (51.375887, 7.660134)
    dest_point = (51.380709, 7.671346)
    
    Ausgabebeispiel:
    0.9450082401845116
    
    """
    
    # Erdradius
    R = 6371.0
    
    # Umrechnung von Gradmaß in Bogenmaß
    lat1 = radians(orig_point[0])
    lon1 = radians(orig_point[1])
    lat2 = radians(dest_point[0])
    lon2 = radians(dest_point[1])
    
    # Berechnung der Distanz
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


def calculate_center(actions):
    """
    Gibt den zentralen Punkt der übergebenen Punkte als Liste zurück.
    Dieser Punkt repräsentiert den Durchschnitt der durch das Tracking-System gesammelten Punkte.
    
    Eingabebeispiel:
    actions = [[51.37459, 7.697596], [51.374501, 7.697512], [51.374204, 7.69759]]
    
    Ausgabebeispiel:
    [51.37443166667289, 7.6975660000324595]
    
    """
    
    x = y = z = 0
    for action in actions:
        # Umrechnung von Gradmaß in Bogenmaß
        lat = radians(action[0])
        lon = radians(action[1])
        # Summe der kartesischen Koordianten bilden
        x += cos(lat) * cos(lon)
        y += cos(lat) * sin(lon)
        z += sin(lat)

    #Durchschnitt der kartesischen Koordianten bilden
    x /= len(actions)
    y /= len(actions)
    z /= len(actions)

    #Umrechnung der kartesischen Koordianten in Bogenmaß und anschließend zurück in Gradmaß (Breiten- und Längengrade)
    lon = atan2(y, x) * 180/pi
    hyp = sqrt(x * x + y * y)
    lat = atan2(z, hyp) * 180/pi
    
    return (lat, lon)
    

In [ ]:
from collections import defaultdict
import mysql.connector as mariadb

host = ''
user = 'root'
password = ''

def get_events():
    """
    Gibt alle events aus der Datenbank als Dictionary zurück
    
    Ausgabebeispiel: {'1': [(51.37459, 7.697596), (51.374501, 7.697512), (51.374204, 7.69759)], '2': [(51.380709, 7.671346)]})
    
    """
    
    mariadb_connection = mariadb.connect(host=host, user=user, password=password, database='disposition')
    cursor = mariadb_connection.cursor()
        
    events = defaultdict(list)

    try:
        # SQL-Abfrage
        cursor.execute("SELECT id, assetuuid, latitude, longitude FROM event")
        
        # Iteration durch das Ergebnis der SQL-Abfrage
        for ident, assetuuid, latitude, longitude in cursor:
            if latitude and longitude:
            # Schreibe Koordianten des aktuellen Events in ein Tupel
                location = (latitude, longitude)
                events[assetuuid].append(location)
            # Wenn das Event gültig ist, füge sie dem Dictionary zum entsprechenden Behälter hinzu
            else:
                print('ungültiger Event-Datensatz. Event-ID: ' + str(ident))

    except mariadb.Error as error:
        print("Error: {}".format(error))
        
    return events


def get_assets():
    """
    Gibt alle assets aus der Datenbank als Dictionary zurück
    
    Ausgabebeispiel: {2: '1', 4: '2'}
    
    """
    
    mariadb_connection = mariadb.connect(host=host, user=user, password=password, database='asset')
    cursor = mariadb_connection.cursor()
    
    assets = dict()

    try:
        # SQL-Abfrage
        cursor.execute("SELECT uuid, address_id, location_id FROM asset")
        
        # Iteration durch das Ergebnis der SQL-Abfrage
        for uuid, address_id, location_id in cursor:
            if location_id:
                assets[location_id] = uuid
                
    except mariadb.Error as error:
        print("Error: {}".format(error))
 
    return assets


def get_locations():
    """
    Gibt alle locations aus der Datenbank als Dictionary zurück
    
    Ausgabebeispiel: {'1': (51.374569, 7.697542), '2': (51.374549, 7.697657)}
    
    """
    
    mariadb_connection = mariadb.connect(host=host, user=user, password=password, database='asset')
    cursor = mariadb_connection.cursor()
    
    assets = get_assets()
    locations = dict()

    try:
        # SQL-Abfrage
        cursor.execute("SELECT id, latitude, longitude FROM address")
        
        # Iteration durch das Ergebnis der SQL-Abfrage
        for location_id, latitude, longitude in cursor:
            if location_id in assets:
                locations[assets[location_id]] = (latitude, longitude)

    except mariadb.Error as error:
        print("Error: {}".format(error))

    return locations

In [ ]:
def get_real():
    """
    Gibt ein Dictionary zurück, das für jeden Behälter eine zentrale Position in einem Tupel beinhaltet.
    Die zentrale Position repräsentiert alle Positionen,
    die für einen Behälter in der Tabelle disposition.events gespeichert sind.
    
    Ausgabebeispiel:
    real = {'1': (51.37443166667289, 7.6975660000324595), '2': (51.380709, 7.671346)}
    
    """
    
    real = dict()
    events = get_events()
    # Bestimme aus den Events einen zentralen Punkt für jeden Behälter
    for key, value, in events.items():
        # Wenn nur eine Position für den Behälter gepeichet ist, übernehme diese
        if len(value) < 2:
            real[key] = value[0]
        # Wenn mindestens zwei Positionen für den Behälter gespeichert sind, berechne ihr Zentrum
        else:
            real[key] = calculate_center(value)
    return real

In [ ]:
def get_coordinates():
    """
    Gibt eine Liste zurück, in der die Breiten- und Längengrade mit folgender Struktur gespeichert werden:
    Für jeden Container gibt es ein Tupel, in dem zwei Positionen gespeichert werden.
    1. Position ist die erwartete, wo der Behälter stehen sollte.
    2. Position ist die tatsächliche, wo sie laut GPS geortet wurde.
    Für jede Position gibt es ein Tupel, in dem Breiten- und Längengrad gespeichert werden.
    
    Ausgabebeispiel:
    coordinates = [((51.374569, 7.697542), (51.37443166667289, 7.6975660000324595)), ((51.374549, 7.697657), (51.380709, 7.671346))]
    
    """
    
    real = get_real()
    locations = get_locations()
    coordinates = list()
    
    for key, value in real.items():
        coordinates.append((value, locations[key]))
        
    return coordinates
    

In [ ]:
import matplotlib.pyplot as plt

def get_figure(coordinates):
    """
    Gibt eine Matplotlib-Figure zurück, auf der die übergebenen Behälterkoordinaten dargestellt werden sollen.
    
    Hinzufügen der Behälterpositionen
    Wenn der Abstand der Behälter größer als 100 Fuß / 30,48 Meter ist, werden sie rot dargestellt
    Wenn der Abstand der Behälter kleiner als 100 Fuß / 30,48 Meter ist, werden sie grün dargestellt
    
    Eingabebeispiel:
    coordinates = [((51.374569, 7.697542), (51.37443166667289, 7.6975660000324595)), ((51.374549, 7.697657), (51.380709, 7.671346))]
    
    Ausgabe: Matplotlib-Figure
    
    """
    
    fig = plt.figure()

    for container in coordinates:
        lats = [container[0][0], container[1][0]]
        lons = [container[0][1], container[1][1]]
        if calculate_distance(container[0], container[1]) > 0.0348:
            plt.plot(lons, lats, 'r') # rote Linie
            plt.plot(lons, lats, 'rs') # rotes Kästchen
        else:
            plt.plot(lons, lats, 'g') # grüne Linie
            plt.plot(lons, lats, 'gs') # grünes Kästchen
            
    return fig

In [ ]:
import mplleaflet

# Erzeugen der Matplotlib-Figure, die auf der OSM projeziert wird
mplleaflet.show(fig=get_figure(get_coordinates())) # Darstellung in neuem Tab
#mplleaflet.display(fig=get_figure(get_coordinates())) # Darstellung im Notebook